In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

In [ ]:
print(torch.cuda.get_device_name(0))


Tesla T4


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(
        policy=transforms.AutoAugmentPolicy.CIFAR10
    ),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_dataset = datasets.CIFAR10(
    root="data",
    train=True,
    transform=transform_train,
    download=True
)

test_dataset = datasets.CIFAR10(
    root="data",
    train=False,
    transform=transform_test,
)

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False
)

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)

torch.Size([128, 3, 32, 32])
torch.Size([128])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=5e-4
)

In [ ]:
images, labels = images.to(device), labels.to(device)

outputs = model(images)
print(outputs.shape)

torch.Size([128, 10])


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    correct = 0
    total = 0
    running_loss = 0.0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return running_loss / len(loader), accuracy

In [ ]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100. * correct / total
    return accuracy

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20
)

In [ ]:
best_acc = 0


num_epochs = 20

for epoch in range(num_epochs):


    model.train()
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion
    )


    model.eval()
    test_acc = evaluate(model, test_loader)

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Acc:  {train_acc:.2f}%")
    print(f"Test Acc:   {test_acc:.2f}%")


    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "best_resnet18.pth")
        print(" Best model saved!")


    scheduler.step()


print("\n Training Complete")
print(f"Best Test Accuracy: {best_acc:.2f}%")


Epoch [1/20]
Train Loss: 0.5584
Train Acc:  80.52%
Test Acc:   84.76%
 Best model saved!

Epoch [2/20]
Train Loss: 0.5521
Train Acc:  80.77%
Test Acc:   85.32%
 Best model saved!

Epoch [3/20]
Train Loss: 0.5387
Train Acc:  81.33%
Test Acc:   85.37%
 Best model saved!

Epoch [4/20]
Train Loss: 0.5187
Train Acc:  82.07%
Test Acc:   85.67%
 Best model saved!

Epoch [5/20]
Train Loss: 0.5153
Train Acc:  82.20%
Test Acc:   85.73%
 Best model saved!

Epoch [6/20]
Train Loss: 0.5112
Train Acc:  82.16%
Test Acc:   85.73%

Epoch [7/20]
Train Loss: 0.5095
Train Acc:  82.40%
Test Acc:   85.84%
 Best model saved!

Epoch [8/20]
Train Loss: 0.5058
Train Acc:  82.43%
Test Acc:   85.75%

Epoch [9/20]
Train Loss: 0.5124
Train Acc:  82.23%
Test Acc:   85.69%

Epoch [10/20]
Train Loss: 0.5328
Train Acc:  81.46%
Test Acc:   84.99%

Epoch [11/20]
Train Loss: 0.5924
Train Acc:  79.24%
Test Acc:   83.79%

Epoch [12/20]
Train Loss: 0.6635
Train Acc:  76.91%
Test Acc:   81.94%

Epoch [13/20]
Train Loss: 0.73

In [ ]:
print(device)

cuda


In [ ]:
torch.save(model.state_dict(), "resnet18_85_8.pth")